In [8]:
import json
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

In [96]:
location= (
            pd.read_excel(r'C:\Users\joaki\Google Drive\dnd\Icewind Dale\Icewind Dale v2.xlsx',sheet_name='location', delimiter = ';')
            .fillna('')
            .assign( Location_name=lambda x: (""+x['Location1']+" "+x['Location2']+"").str.replace(" ","_"),
                     Main_location_name=lambda x: (""+x['Location1']+"").str.replace(" ","_"),
                     Location_nr=lambda x: x.groupby('Main_location_name').ngroup(ascending=True)+1
                   )
            .sort_values(by='Main_location_name')
            .assign(Location1_nr = lambda x: x.groupby('Main_location_name').cumcount()+1,
                    Row = np.arange(len(location))+1)
 
)

location.to_json(r'C:\Users\joaki\Google Drive\dnd\Icewind Dale\Icewind-Dale\location.json',orient='records')



person = (
            pd.read_excel(r'C:\Users\joaki\Google Drive\dnd\Icewind Dale\Icewind Dale v2.xlsx',sheet_name='person', delimiter = ';')
              .assign(Location2	 = lambda x: x['Location2'].fillna(''),
                      Main_location_name=lambda x: (""+x['Location1']+" "+x['Location2']+"").str.replace(" ","_"))
              .sort_values(by = ['Main_location_name','First_Name'])
              .assign(Location_nr=lambda x: x.groupby('Main_location_name').ngroup(ascending=True)+1,
                      Location1_nr=lambda x: x.groupby('Location1').cumcount()+1,
                      Location2_nr=lambda x: x.groupby('Main_location_name').cumcount()+1,
                      Row= lambda x:  np.arange(len(x)),
                      Surname= lambda x: x['Surname'].fillna(''),
                      Party_relationship= lambda x: x['Party_relationship'].fillna(''),
                      ID=lambda x: "ID"+x['First_Name']+"",
                      full_name=lambda x: x['First_Name']+"_"+x['Surname']+"")
              .sort_values(by = 'Column')
              .assign(First_faction=lambda x: (x.groupby('Column').cumcount()+1 ).apply(lambda x: 1 if x==1 else 0),
                      Faction_id=lambda x: (""+x['Faction_name']+"").str.replace(" ","_"),
                      First_alignment=lambda x:(x.groupby('Column').cumcount()+1).apply(lambda x: 1 if x==1 else 0)
                     )
              .sort_values(by = ['Main_location_name','First_Name'])       
               
    
                     )

quest= (
        pd.read_excel(r'C:\Users\joaki\Google Drive\dnd\Icewind Dale\Icewind Dale v2.xlsx',sheet_name='quest', delimiter = ';')
          .fillna('')
          .assign(Main_location_name=lambda x: (""+x['Location1']+"").str.replace(" ","_"),
                  Location_name=lambda x: (""+x['Location1']+" "+x['Location2']+"").str.replace(" ","_"),
                  Row = np.arange(len(quest))+1
                  )
        )
quest.to_json(r'C:\Users\joaki\Google Drive\dnd\Icewind Dale\Icewind-Dale\quest.json',orient='records')  



quest_giver= ( quest
             .query("Completed==0" )
             .assign(test = lambda x: x['Quest_giver'].str.len())
             .query("test!=0")

             .rename(columns = {'Quest_giver': 'quest_giver_name'}, inplace = False)
             .assign(quest_giver=1,
                     quest_giver_underline=lambda x: (""+x['quest_giver_name']+"").str.replace(" ","_"))  
             .loc[:,['quest_giver_name','quest_giver','quest_giver_underline']]
#            .loc[:,['quest_giver_name','quest_giver','quest_giver_underline']]
  
)

quest_target= ( quest
            .query("Completed==0" )
             .assign(test = lambda x: x['Quest_target'].str.len())
             .query("test!=0")
             .rename(columns = {'Quest_target': 'quest_target_name'}, inplace = False)
             .assign(quest_target=1,
                     quest_target_underline=lambda x: (""+x['quest_target_name']+"").str.replace(" ","_"))        
             .loc[:,['quest_target_name','quest_target','quest_target_underline']]
  
)


# person.to_json(r'C:\Users\joaki\Google Drive\dnd\Icewind Dale\Icewind-Dale\person.json',orient='records')
person = (
             pd.merge(person, quest_target,how='left', left_on='full_name', right_on='quest_target_underline')
            .drop(['quest_target_name','quest_target_underline'], axis=1)    
 )

person = (
             pd.merge(person, quest_giver,how='left', left_on='full_name', right_on='quest_giver_underline')
            .drop(['quest_giver_name','quest_giver_underline'], axis=1)
            .assign(quest_target = lambda x: x['quest_target'].fillna('0'),
                    quest_giver = lambda x: x['quest_giver'].fillna('0'),
                   )
            .sort_values(by = ['Main_location_name','First_Name'])
 )

person.to_json(r'C:\Users\joaki\Google Drive\dnd\Icewind Dale\Icewind-Dale\person.json',orient='records')  


In [97]:
person

,Location1,Location2,Surname,First_Name,Column,Alignment,Faction_name,Deceased,Race,Occupation,Party_relationship,Main_location_name,Location_nr,Location1_nr,Location2_nr,Row,ID,full_name,First_faction,Faction_id,First_alignment,quest_target,quest_giver
0,Bryn Shander,Northlook,,Clinn,2,Unaligned,Unassociated,0,Shield Dwarf,Retired Bounty hunter,quest giver,Bryn_Shander_Northlook,1,1,1,0,IDClinn,Clinn_,1,Unassociated,1,0,0
1,Bryn Shander,Temple of the morninglord,,?,2,Unaligned,Unassociated,0,Gnome,Clericof Lathander,,Bryn_Shander_Temple_of_the_morninglord,2,2,1,1,ID?,?_,0,Unassociated,0,0,0
2,Bryn Shander,Temple of the morninglord,,Lashan,2,Unaligned,Unassociated,0,Unknown,Unknown,,Bryn_Shander_Temple_of_the_morninglord,2,3,2,2,IDLashan,Lashan_,0,Unassociated,0,0,0
3,Bryn Shander,The black iron blades,,Elsa,2,Unaligned,Unassociated,0,Dwarf,Shop owner,,Bryn_Shander_The_black_iron_blades,3,4,1,3,IDElsa,Elsa_,0,Unassociated,0,0,0
4,Bryn Shander,The black iron blades,The Hammer,Garn,2,Unaligned,Unassociated,0,Dwarf,Smith,,Bryn_Shander_The_black_iron_blades,3,5,2,4,IDGarn,Garn_The Hammer,0,Unassociated,0,0,0
5,Bryn Shander,Unknown,,Freddo,2,Unaligned,Unassociated,0,Halfling,Unknown,,Bryn_Shander_Unknown,4,6,1,5,IDFreddo,Freddo_,0,Unassociated,0,0,0
6,Bryn Shander,Unknown,,Hruna,2,Unaligned,Unassociated,0,Dwarf,Unknown,,Bryn_Shander_Unknown,4,7,2,6,IDHruna,Hruna_,0,Unassociated,0,0,0
7,Bryn Shander,Unknown,,Khorox,2,Unaligned,Unassociated,0,Dwarf,Unknown,,Bryn_Shander_Unknown,4,8,3,7,IDKhorox,Khorox_,0,Unassociated,0,0,0
8,Bryn Shander,Unknown,,Storn,2,Unaligned,Unassociated,0,Dwarf,Unknown,,Bryn_Shander_Unknown,4,9,4,8,IDStorn,Storn_,0,Unassociated,0,0,0
9,Unknown,,Kaltro,Sephek,2,Unaligned,Unassociated,0,Unknown,Body guard of Torrga,target,Unknown_,5,1,1,9,IDSephek,Sephek_Kaltro,0,Unassociated,0,1,0


In [81]:
person

,Location1,Location2,Surname,First_Name,Column,Alignment,Faction_name,Deceased,Race,Occupation,Party_relationship,Main_location_name,Location_nr,Location1_nr,Location2_nr,Row,ID,full_name,First_faction,Faction_id,First_alignment,quest_target,quest_giver
0,Bryn Shander,Northlook,,Clinn,2,Unaligned,Unassociated,0,Shield Dwarf,Retired Bounty hunter,quest giver,Bryn_Shander_Northlook,1,1,1,0,IDClinn,Clinn_,1,Unassociated,1,0,0
1,Bryn Shander,Temple of the morninglord,,?,2,Unaligned,Unassociated,0,Gnome,Clericof Lathander,,Bryn_Shander_Temple_of_the_morninglord,2,2,1,1,ID?,?_,0,Unassociated,0,0,0
2,Bryn Shander,Temple of the morninglord,,Lashan,2,Unaligned,Unassociated,0,Unknown,Unknown,,Bryn_Shander_Temple_of_the_morninglord,2,3,2,2,IDLashan,Lashan_,0,Unassociated,0,0,0
3,Bryn Shander,The black iron blades,,Elsa,2,Unaligned,Unassociated,0,Dwarf,Shop owner,,Bryn_Shander_The_black_iron_blades,3,4,1,3,IDElsa,Elsa_,0,Unassociated,0,0,0
4,Bryn Shander,The black iron blades,The Hammer,Garn,2,Unaligned,Unassociated,0,Dwarf,Smith,,Bryn_Shander_The_black_iron_blades,3,5,2,4,IDGarn,Garn_The Hammer,0,Unassociated,0,0,0
5,Bryn Shander,Unknown,,Freddo,2,Unaligned,Unassociated,0,Halfling,Unknown,,Bryn_Shander_Unknown,4,6,1,5,IDFreddo,Freddo_,0,Unassociated,0,0,0
6,Bryn Shander,Unknown,,Hruna,2,Unaligned,Unassociated,0,Dwarf,Unknown,,Bryn_Shander_Unknown,4,7,2,6,IDHruna,Hruna_,0,Unassociated,0,0,0
7,Bryn Shander,Unknown,,Khorox,2,Unaligned,Unassociated,0,Dwarf,Unknown,,Bryn_Shander_Unknown,4,8,3,7,IDKhorox,Khorox_,0,Unassociated,0,0,0
8,Bryn Shander,Unknown,,Storn,2,Unaligned,Unassociated,0,Dwarf,Unknown,,Bryn_Shander_Unknown,4,9,4,8,IDStorn,Storn_,0,Unassociated,0,0,0
9,Unknown,,Kaltro,Sephek,2,Unaligned,Unassociated,0,Unknown,Body guard of Torrga,target,Unknown_,5,1,1,9,IDSephek,Sephek_Kaltro,0,Unassociated,0,1,0
